## 여기어때 숙소 정보 크롤러
--- 
**기능**
1. 사용자가 지정한 여행지, 날짜, 인원으로 숙소를 검색합니다.
2. 검색 결과 페이지에 나타난 숙소 목록의 이름, 가격, 평점 등을 수집합니다.
3. 결과를 `(위치)_yeogi.csv` csv 파일과 `yeogi.ipynb` 파일로 저장합니다.

**필요 라이브러리 설치**
```bash
pip install selenium webdriver-manager beautifulsoup4 pandas 등
```

In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

#### 검색 결과창에서 크롤링
- 숙소명, 숙소유형, 위치, 평점, 가격 뽑아서
- csv 형식으로 저장

In [ ]:
# 검색 조건 설정
keyword = "서울+홍대"
checkin = "2025-09-06"
checkout = "2025-09-15"
adult = 4  

page = 1
results = []

# 크롬 옵션 설정
options = Options()
options.add_argument("--window-size=1200,800")  # 창 크기 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 페이지 반복 크롤링
while True:
    url = f"https://www.yeogi.com/domestic-accommodations?keyword={keyword}&checkIn={checkin}&checkOut={checkout}&personal={adult}&freeForm=false&page={page}"
    driver.get(url)
    time.sleep(3)  # 페이지 로딩 대기

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 각 요소 선택자
    name_tags = soup.select("h3.gc-thumbnail-type-seller-card-title")
    type_tags = soup.select("ul.css-bl7zf6 li:first-child")
    location_blocks = soup.select("div.css-19li9i9")
    rating_tags = soup.select("div.css-19f645y span.css-9ml4lz")
    price_tags = soup.select("div.css-yeouz0")

    n = min(len(name_tags), len(type_tags), len(location_blocks), len(rating_tags), len(price_tags))

    if n == 0:
        print(f"[INFO] Page {page}: 숙소 없음, 종료.")
        break

    print(f"[INFO] Page {page}: 숙소 {n}개 크롤링 중...")

    # 각 숙소 정보 저장
    for i in range(n):
        name = name_tags[i].text.strip()
        acc_type = type_tags[i].text.strip()
        location = " ".join([span.text.strip() for span in location_blocks[i].select("span")])
        rating = rating_tags[i].text.strip()
        price_tag = price_tags[i]

        if price_tag:
            price_parts = [span.get_text(strip=True) for span in price_tag.select("span")]
            price_text = " ".join(price_parts).replace(" 원", "원").replace(" /", "/")
        else:
            price_text = "없음"

        results.append([name, acc_type, location, rating, price_text])

    page += 1

driver.quit()

# CSV 파일로 저장
with open("hongdae_accommodations.csv", "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["이름", "유형", "위치", "평점", "가격"])  # 헤더
    writer.writerows(results)

print("[완료] CSV 파일로 숙소 정보 저장 완료.")

[INFO] Page 1: 숙소 7개 크롤링 중...
[INFO] Page 2: 숙소 없음, 종료.
[완료] CSV 파일로 숙소 정보 저장 완료.


#### 함수로 선언

In [ ]:
def crawl_yeogi_to_csv(destination: str, checkin: str, checkout: str, adults: int) -> str:
    keyword = destination.replace(" ", "+")
    filename = destination.replace(" ", "_") + "_yeogi.csv"

    page = 1
    results = []

    options = Options()
    options.add_argument("--window-size=1200,800")
    options.add_argument("--headless=new")  # UI 없이 실행 (선택사항)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    while True:
        url = f"https://www.yeogi.com/domestic-accommodations?keyword={keyword}&checkIn={checkin}&checkOut={checkout}&personal={adults}&freeForm=false&page={page}"
        driver.get(url)
        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        name_tags = soup.select("h3.gc-thumbnail-type-seller-card-title")
        type_tags = soup.select("ul.css-bl7zf6 li:first-child")
        location_blocks = soup.select("div.css-19li9i9")
        rating_tags = soup.select("div.css-19f645y span.css-9ml4lz")
        price_tags = soup.select("div.css-yeouz0")

        n = min(len(name_tags), len(type_tags), len(location_blocks), len(rating_tags), len(price_tags))

        if n == 0:
            print(f"[INFO] Page {page}: 숙소 없음, 종료.")
            break

        print(f"[INFO] Page {page}: 숙소 {n}개 크롤링 중...")

        for i in range(n):
            name = name_tags[i].text.strip()
            acc_type = type_tags[i].text.strip()
            location = " ".join([span.text.strip() for span in location_blocks[i].select("span")])
            rating = rating_tags[i].text.strip()
            price_tag = price_tags[i]

            if price_tag:
                price_parts = [span.get_text(strip=True) for span in price_tag.select("span")]
                price_text = " ".join(price_parts).replace(" 원", "원").replace(" /", "/")
            else:
                price_text = "없음"

            results.append([name, acc_type, location, rating, price_text])

        page += 1

    driver.quit()

    with open(filename, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["이름", "유형", "위치", "평점", "가격"])
        writer.writerows(results)

    print(f"[완료] CSV 저장: {filename}")
    return filename

In [3]:
# 광안리 지역 2025년 9월 6일 ~ 15일, 2인
csv_path = crawl_yeogi_to_csv("부산 광안리", "2025-09-06", "2025-09-15", 2)

[INFO] Page 1: 숙소 20개 크롤링 중...
[INFO] Page 2: 숙소 20개 크롤링 중...
[INFO] Page 3: 숙소 10개 크롤링 중...
[INFO] Page 4: 숙소 없음, 종료.
[완료] CSV 저장: 부산_광안리_yeogi.csv
